In [73]:
!pip install datasets
!pip install openpyxl
!pip install -q -U google-genai
# !pip install transformers
# !pip install accelerate
# !pip install peft
# !pip install bitsandbytes

# Imports

In [74]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import sys

# Call models
from src.call_models import bedrock_connect, call_claude_bedrock
from src.call_models import google_connect, call_gemini, all_string_gemini_config, all_int_gemini_config
from src.translate_func import claude_translation, gemini_translation

# Datasets
from src.benchmarks_code import arc_ai
from src.benchmarks_code import gsm8k
from src.benchmarks_code import hellaswag
# from prompts import arc_prompts
from prompts import gsm_prompts
# from prompts import hellaswag_prompts

# Access keys
from my_access_keys import google_access_key, aws_access_key, aws_secret_key

# .csv utils
from src.save_utils import add_dataset_to_csv

# Remove annoying warning
from IPython.core.display_functions import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
# Get the model's access keys
bedrock_client = bedrock_connect(aws_access_key, aws_secret_key)
google_client = google_connect(google_access_key)

# Check Models Calls

In [76]:
print('Gemini:')
generate_content_config = all_string_gemini_config(['recipe', 'ingredients'], 'ALWAYS THINK IN HEBREW BEFORE ANSWERING!', think_bud=200)
response = call_gemini(google_client, "List a popular cookie recipe, and include the amounts of ingredients.", generate_content_config)
print(response.text)
print('---')
my_recipes = response.parsed
print(my_recipes.keys())

Gemini:
{"recipe":"Chocolate Chip Cookies","ingredients":"2 1/4 cups all-purpose flour, 1 teaspoon baking soda, 1 teaspoon salt, 1 cup (2 sticks) unsalted butter softened, 3/4 cup granulated sugar, 3/4 cup packed brown sugar, 1 teaspoon vanilla extract, 2 large eggs, 2 cups (12 ounces) semi-sweet chocolate chips"}
---
dict_keys(['recipe', 'ingredients'])


In [5]:
print('Gemini:')
generate_content_config = all_string_gemini_config(['recipe', 'ingredients', 'thoughts_process'], 'ALWAYS THINK IN HEBREW BEFORE ANSWERING!', think_bud=200)
response = call_gemini(google_client, "תתאר מתכון פופולרי לעוגיות ותוסיך את הכמויות של כל מרכיב. תסביר את דרך המחשבה שלך בעברית צעד אחרי צעד.", generate_content_config)
print(response.text)
print('---')
my_recipes = response.parsed
print(my_recipes.keys())

Gemini:
{"recipe":"עוגיות שוקולד צ'יפס קלאסיות","ingredients":"2 ורבע כוסות קמח לבן; 1 כפית סודה לשתייה; 1 כפית מלח; 1 כוס (226 גרם) חמאה ללא מלח, מרוככת; 3/4 כוס סוכר לבן; 3/4 כוס סוכר חום דחוס; 2 ביצים גדולות; 1 כפית תמצית וניל; 2 כוסות שוקולד צ'יפס (חלב, מריר או שילוב)","thoughts_process":"בחרתי במתכון לעוגיות שוקולד צ'יפס קלאסיות מכיוון שהן נחשבות לאחד מסוגי העוגיות הפופולריים והאהובים ביותר בעולם. הן קלות להכנה ומתאימות לכל אירוע.הכמויות נבחרו על בסיס מתכון סטנדרטי ומוכח לעוגיות שוקולד צ'יפס, המבטיח מרקם וטעם אופטימליים. הקפדתי לכלול את כל המרכיבים החיוניים: קמח, חומר התפחה (סודה לשתייה), מלח לאיזון טעמים, שומן (חמאה) למרקם עשיר, סוכרים (לבן וחום) למתיקות וללחות, ביצים לקישור, וניל לטעם, וכמובן שוקולד צ'יפס כמרכיב המרכזי.ההסבר מפורט מספיק כדי שכל אחד יוכל להבין את המתכון וליישם אותו.השפה היא עברית כפי שנדרש.הפורמט הוא JSON תקין, עם כל השדות הנדרשים: recipe, ingredients, ו-thoughts_process."}
---
dict_keys(['recipe', 'ingredients', 'thoughts_process'])


# ARC_AI2

## Get Dataset

In [7]:
arc_dataset = arc_ai.get_arc_ai2_datasets()
arc_dataset['arc_challenge_train'] = arc_dataset['arc_challenge_train'].skip(5).take(50)
file_name = 'compare_csv/arc_ai2_chall_train_top_50.csv'

print(file_name)
arc_dataset

README.md: 0.00B [00:00, ?B/s]

ARC-Challenge/train-00000-of-00001.parqu(…):   0%|          | 0.00/190k [00:00<?, ?B/s]

ARC-Challenge/test-00000-of-00001.parque(…):   0%|          | 0.00/204k [00:00<?, ?B/s]

ARC-Challenge/validation-00000-of-00001.(…):   0%|          | 0.00/55.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1172 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/299 [00:00<?, ? examples/s]

compare_csv/arc_ai2_chall_train_top_50.csv


{'arc_challenge_train': Dataset({
     features: ['id', 'question', 'choices', 'answerKey'],
     num_rows: 50
 })}

In [9]:
df = pd.read_csv(file_name)
text_df = pd.read_csv(file_name[:-4] + '-text.csv')
df.head(3)

,original,claude_v1,claude_v2,claude_v3,claude_v4,claude_v5,claude_v6,gemini_v1,gemini_pro_v1,gemini_v2,gemini_pro_v2,gemini_v2_think,gemini_v3,gemini_v3_think,gemini_pro_v1_think,gemini_v1_think,claude_v6_system_prompt,gemini_v1_few_options,claude_v1_few_options
0,<question>Which land form is the result of the...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזה צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...
1,<question>Which statement best compares single...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזו טענה משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...
2,"<question>As part of an experiment, an astrona...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח מאזניים ...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר...","<question>במסגרת ניסוי, אסטרונאוט לוקח מאזניים...","<question>כחלק מניסוי, אסטרונאוט לוקח משקל ליר..."


In [10]:
text_df.head(2)

,original,claude_v1_few_options text
0,<question>Which land form is the result of the...,I'll translate the question and answer options...
1,<question>Which statement best compares single...,I'll translate the English question and answer...


## Run Translation

### Claude

In [16]:
exp_name = 'claude_v1_refine'

hebrew_datasets, text_output = claude_translation(
    bedrock_client,
    arc_dataset,
    arc_prompts.ARC_INSTRUCT_V1_CLAUDE_REFINE,
    arc_prompts.ARC_FEW_SHOTS,
    arc_prompts.ARC_FORMAT,
    arc_ai.arc_sample_to_dict,
    arc_ai.arc_dict_to_sample,
)

Translating arc_challenge_train...


  0%|          | 0/50 [00:00<?, ?it/s]

In [17]:
df = add_dataset_to_csv(file_name, exp_name, hebrew_datasets['arc_challenge_train'], arc_ai.arc_sample_to_dict)
df.head(2)

,original,claude_v1,claude_v2,claude_v3,claude_v4,claude_v5,claude_v6,gemini_v1,gemini_pro_v1,gemini_v2,gemini_pro_v2,gemini_v2_think,gemini_v3,gemini_v3_think,gemini_pro_v1_think,gemini_v1_think,claude_v6_system_prompt,gemini_v1_few_options,claude_v1_few_options,claude_v1_refine
0,<question>Which land form is the result of the...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזה צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...
1,<question>Which statement best compares single...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזו טענה משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...


In [18]:
# text_df = df[['original']].copy()
text_df[exp_name + ' text'] = text_output['arc_challenge_train']
text_df.head(2)

,original,claude_v1_few_options text,claude_v1_refine text
0,<question>Which land form is the result of the...,I'll translate the question and answer options...,First translation:\n<question>איזו צורת קרקע ה...
1,<question>Which statement best compares single...,I'll translate the English question and answer...,First translation:\n<question>איזה היגד משווה ...


In [19]:
text_df.to_csv(file_name[:-4] + '-text.csv', index=False)

### Gemini

In [64]:
exp_name = 'gemini_v1_few_options'

hebrew_datasets = gemini_translation(
    google_client,
    arc_dataset,
    arc_prompts.ARC_INSTRUCT_V1_GEMINI_MULTI,
    arc_prompts.ARC_FEW_SHOTS,
    arc_ai.arc_sample_to_dict,
    arc_ai.arc_dict_to_sample,
    if_pro=False,
    think_bud=20_000
)

Translating arc_challenge_train...


  0%|          | 0/50 [00:00<?, ?it/s]

Sleeping.... Done!                                

In [65]:
hebrew_datasets

{'arc_challenge_train': Dataset({
     features: ['id', 'question', 'choices', 'answerKey'],
     num_rows: 50
 })}

In [66]:
df = add_dataset_to_csv(file_name, exp_name, hebrew_datasets['arc_challenge_train'], arc_ai.arc_sample_to_dict)
df.head(2)

,original,claude_v1,claude_v2,claude_v3,claude_v4,claude_v5,claude_v6,gemini_v1,gemini_pro_v1,gemini_v2,gemini_pro_v2,gemini_v2_think,gemini_v3,gemini_v3_think,gemini_pro_v1_think,gemini_v1_think,claude_v6_system_prompt,gemini_v1_few_options
0,<question>Which land form is the result of the...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת נוף היא תוצאה של כוח בונה ...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו תצורת נוף היא תוצאה של הכוח הבו...,<question>איזו צורת נוף היא תוצאה של הכוח הבונ...,<question>איזו צורת קרקע היא תוצאה של הכוח הבו...,<question>איזה צורת נוף היא תוצאה של כוח בונה ...
1,<question>Which statement best compares single...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזו טענה משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר או...,<question>איזה היגד משווה בצורה הטובה ביותר בי...,<question>איזה היגד משווה בצורה הטובה ביותר בי...


### Multi-options Translation - Gemini

### Claude vs Gemini (using Gemini as judge)

# GSM8K

## Get Dataset

In [77]:
gsm_dataset = gsm8k.get_gsm8k_datasets()
gsm_dataset['gsm8k_train'] = gsm_dataset['gsm8k_train'].skip(5).take(50)
gsm_file_name = 'compare_csv/gsm8k_main_train_top_50.csv'

print(gsm_file_name)
gsm_dataset

compare_csv/gsm8k_main_train_top_50.csv


{'gsm8k_train': Dataset({
     features: ['question', 'answer'],
     num_rows: 50
 })}

In [78]:
def only_keep_final_score(sample):
    sample['answer'] = sample['answer'].split('####')[-1].strip()
    return sample

gsm_dataset['gsm8k_train_final'] = gsm_dataset['gsm8k_train'].map(only_keep_final_score)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [79]:
gsm_dataset['gsm8k_train_final']['question']

Column(['Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?', 'Albert is wondering how much pizza he can eat in one day. He buys 2 large pizzas and 2 small pizzas. A large pizza has 16 slices and a small pizza has 8 slices. If he eats it all, how many pieces does he eat that day?', 'Ken created a care package to send to his brother, who was away at boarding school.  Ken placed a box on a scale, and then he poured into the box enough jelly beans to bring the weight to 2 pounds.  Then, he added enough brownies to cause the weight to triple.  Next, he added another 2 pounds of jelly beans.  And finally, he added enough gummy worms to double the weight once again.  What was the final weight of the box of goodies, in pounds?', 'Alexis is applying for a new job and bou

In [80]:
gsm_dataset['gsm8k_train'][0]['answer'].split('####')[-1].strip()

'35'

In [ ]:
df = add_dataset_to_csv(gsm_file_name, 'original', gsm_dataset['gsm8k_train'], gsm8k.gsm8k_sample_to_dict)
text_df = add_dataset_to_csv(gsm_file_name[:-4] + '-text.csv', 'original', gsm_dataset['gsm8k_train'], gsm8k.gsm8k_sample_to_dict)
df.head(3)

In [ ]:
df = pd.read_csv(gsm_file_name)
# text_df = pd.read_csv(gsm_file_name[:-4] + '-text.csv')
display(df.head(2))
# display(text_df.head(2))

## Run Translation

### Claude

In [61]:
exp_name = 'claude_v1_refine'

hebrew_datasets, text_output = claude_translation(
    bedrock_client,
    gsm_dataset,
    gsm_prompts.GSM_INSTRUCT_CLAUDE_REFINE_V1,
    gsm_prompts.GSM_FEW_SHOTS,
    # gsm_prompts.GSM_FORMAT,
    gsm_prompts.GSM_FORMAT_REFINE,
    gsm8k.gsm8k_sample_to_dict,
    gsm8k.gsm8k_dict_to_sample,
)

Translating gsm_main_train...


  0%|          | 0/50 [00:00<?, ?it/s]

In [62]:
def len_mat(text):
    pattern = r"<(?!response_format\b)([^>]+)>(.*?)</\1>"
    matches = re.findall(pattern, text, re.DOTALL)
    return len(matches)

In [63]:
pd.Series([len_mat(i) for i in text_output['gsm8k_train']]).value_counts()

4    50
Name: count, dtype: int64

In [64]:
df = add_dataset_to_csv(gsm_file_name, exp_name, hebrew_datasets['gsm8k_train'], gsm8k.gsm8k_sample_to_dict)
df.head(2)

,original,claude_v1,gemini_pro_v1,claude_v1_refine
0,<question>Mark has a garden with flowers. He p...,<question>למארק יש גינה עם פרחים. הוא שתל בה צ...,<question>למארק יש גינה עם פרחים. הוא שתל בה צ...,<question>למארק יש גינה עם פרחים. הוא שתל צמחי...
1,<question>Albert is wondering how much pizza h...,<question>אלברט תוהה כמה פיצה הוא יכול לאכול ב...,<question>אלברט תוהה כמה פיצה הוא יכול לאכול ב...,<question>אלברט תוהה כמה פיצה הוא יכול לאכול ב...


In [65]:
text_df[exp_name + ' text'] = text_output['gsm8k_train']
text_df.to_csv(gsm_file_name[:-4] + '-text.csv', index=False)
text_df.head(2)

,original,claude_v1 text,gemini_pro_v1 text,claude_v1_refine text
0,<question>Mark has a garden with flowers. He p...,<question>למארק יש גינה עם פרחים. הוא שתל בה צ...,"**Thought Process Summary:**\n\nOkay, here's w...",First translation attempt:\n<question>למארק יש...
1,<question>Albert is wondering how much pizza h...,<question>אלברט תוהה כמה פיצה הוא יכול לאכול ב...,**Pizza Problem: English to Hebrew Translation...,First translation attempt:\n<question>אלברט תו...


In [66]:
for i in range(50):
    print(text_df['claude_v1_refine text'].iloc[i])
    print('\n - - - - - - - - - - - - - - - - - - - - - - - - - - - -\n')

First translation attempt:
<question>למארק יש גינה עם פרחים. הוא שתל צמחים בשלושה צבעים שונים בה. עשרה מהם צהובים, ויש 80% יותר מאלה בסגול. יש רק 25% כמות פרחים ירוקים כמו שיש פרחים צהובים וסגולים. כמה פרחים יש למארק בגינה שלו?</question>
<answer>יש 80/100 * 10 = <<80/100*10=8>>8 יותר פרחים סגולים מאשר פרחים צהובים.
אז בגינה של מארק, יש 10 + 8 = <<10+8=18>>18 פרחים סגולים.
פרחים סגולים וצהובים מסתכמים ל-10 + 18 = <<10+18=28>>28 פרחים.
זה אומר שבגינה של מארק יש 25/100 * 28 = <<25/100*28=7>>7 פרחים ירוקים.
אז בסך הכל למארק יש 28 + 7 = <<28+7=35>>35 צמחים בגינה שלו.
#### 35</answer>

Improved translation:
<question>למארק יש גינה עם פרחים. הוא שתל צמחים בשלושה צבעים שונים. עשרה מהם צהובים, ויש 80% יותר פרחים בצבע סגול. יש רק 25% מכמות הפרחים הצהובים והסגולים בצבע ירוק. כמה פרחים יש למארק בגינה שלו?</question>
<answer>יש 80/100 * 10 = <<80/100*10=8>>8 יותר פרחים סגולים מאשר פרחים צהובים.
אז בגינה של מארק, יש 10 + 8 = <<10+8=18>>18 פרחים סגולים.
פרחים סגולים וצהובים מסתכמים ל-10 + 18 = <<10+

### Gemini

In [81]:
gsm_dataset_no_answer = {}
gsm_dataset_no_answer['gsm8k_train'] = gsm_dataset['gsm8k_train'].map(only_keep_final_score)

In [82]:
exp_name = 'gemini_flash_v1'

hebrew_datasets_synth, text_output = gemini_translation(
    google_client,
    # small,
    gsm_dataset_no_answer,
    gsm_prompts.GSM_SYNTH_INSTRUCT_GEMINI_V1,
    gsm_prompts.GSM_SYNTH_FEW_SHOTS,
    gsm8k.gsm8k_sample_to_dict,
    gsm8k.gsm8k_dict_to_sample,
    if_pro=False,
    think_bud=20_000
)

Translating gsm8k_train...


  0%|          | 0/50 [00:00<?, ?it/s]

In [83]:
exp_name = 'gemini_flash_v1'

hebrew_datasets_translated, text_output = gemini_translation(
    google_client,
    # small,
    gsm_dataset,
    gsm_prompts.GSM_INSTRUCT_GEMINI_V1,
    gsm_prompts.GSM_FEW_SHOTS,
    gsm8k.gsm8k_sample_to_dict,
    gsm8k.gsm8k_dict_to_sample,
    if_pro=False,
    think_bud=20_000
)

Translating gsm8k_train...


  0%|          | 0/50 [00:00<?, ?it/s]

Sleeping.... Done!Translating gsm8k_train_final...


  0%|          | 0/50 [00:00<?, ?it/s]

Sleeping.... Done!                                

In [84]:
df = add_dataset_to_csv('compare_csv/gsm8k_judge_50_flash.csv', 'synth_heb', hebrew_datasets_synth['gsm8k_train'], gsm8k.gsm8k_sample_to_dict)
df = add_dataset_to_csv('compare_csv/gsm8k_judge_50_flash.csv', 'trans_heb', hebrew_datasets_translated['gsm8k_train'], gsm8k.gsm8k_sample_to_dict)
df.head(2)

,synth_heb,trans_heb
0,<question>למארק יש גינה עם פרחים. הוא שתל בה צ...,<question>למארק יש גינה עם פרחים. הוא שתל בה צ...
1,<question>אלברט תוהה כמה פיצה הוא יכול לאכול ב...,<question>אלברט תוהה כמה פיצה הוא יכול לאכול ב...


## LLM-as-a-Judge

In [88]:
import enum

class COT_VERSION(enum.Enum):
  TRANSLATED = "Translated"
  SYNTHESIZED = "Synthesized"


def judge_cot_version(trans: str, synth: str):
    content = f"""Here are two options for an answer.
    In your decision, take into factor:
    1. The correctness of the answer.
    2. The logic flow.
    3. Its coherence.
    4. How much it is didactic.

    <translated>\n{trans}\n<synthesized>\n{synth}"""

    try:
        generate_content_config = all_string_gemini_config(['decision'], 'ALWAYS THINK IN HEBREW BEFORE ANSWERING!', think_bud=200, enum=cot_version)
        response = call_gemini(google_client, content, generate_content_config)
        return response.text
    except Exception as E:
        return str(E)

In [89]:
df = pd.read_csv('compare_csv/gsm8k_judge_50_flash.csv')
df['judge'] = df.apply(lambda r: judge_cot_version(r['trans_heb'], r['synth_heb']), axis=1)
df

,synth_heb,trans_heb,judge
0,<question>למארק יש גינה עם פרחים. הוא שתל בה צ...,<question>למארק יש גינה עם פרחים. הוא שתל בה צ...,"{""decision"": ""Synthesized""}"
1,<question>אלברט תוהה כמה פיצה הוא יכול לאכול ב...,<question>אלברט תוהה כמה פיצה הוא יכול לאכול ב...,"{""decision"":""Synthesized""}"
2,"<question>קן יצר חבילת טיפול לשלוח לאחיו, שהיה...","<question>קן יצר חבילת טיפול לשלוח לאחיו, שהיה...","{""decision"": ""Synthesized""}"
3,<question>אלכסיס מגישה מועמדות לעבודה חדשה וקנ...,<question>אלכסיס מגישה מועמדות לעבודה חדשה וקנ...,"{""decision"": ""Synthesized""}"
4,<question>טינה מרוויחה 18.00$ לשעה. אם היא עוב...,<question>טינה מרוויחה 18.00$ לשעה. אם היא עוב...,"{""decision"": ""Synthesized""}"
5,<question>מפלצת ים עמוקה עולה מהמים אחת למאה ש...,<question>מפלצת ים עמוק עולה מהמים אחת למאה שנ...,"{""decision"": ""Synthesized""}"
6,<question>טוביאס קונה זוג נעליים חדש שעולה 95$...,<question>טוביאס קונה זוג נעליים חדש שעולה 95$...,"{""decision"": ""Synthesized""}"
7,<question>לרנדי יש 60 עצי מנגו בחווה שלו. יש ל...,<question>לרנדי יש 60 עצי מנגו בחווה שלו. יש ל...,"{""decision"": ""Translated""}"
8,<question>ג'ספר יגיש מגש נקניקים וגבינות במסיב...,<question>ג'ספר יגיש מגש נקניקים וגבינות במסיב...,"{""decision"": ""Synthesized""}"
9,<question>ג'וי יכולה לקרוא 8 עמודים מספר ב-20 ...,<question>ג'וי יכולה לקרוא 8 עמודים מספר ב-20 ...,"{""decision"": ""Translated""}"


In [98]:
add_dataset_to_csv('compare_csv/gsm8k_judge_50_flash-1.csv', 'judge', df['judge'].tolist()[:12], lambda x : x)

TypeError: string indices must be integers